Fake ClickHouse Data

In [1]:
import pandas as pd

df_clickhouse_fake = pd.DataFrame({
    "jour": [
        "2024-01-01",
        "2024-01-02",
        "INVALID_DATE"     # volontairement faux pour tester la standardisation
    ],
    "valeur": [
        "10.5",
        "20.0",
        "30.0"
    ],
    "type_vehicule": [
        "SUV",
        "Berline",
        "SUV"
    ],
    "kpi": [
        "Ventes",
        "Ventes",
        "Ventes"
    ]
})


Le fichier standardization.py comporte 2 classes :
- qui définit le format BI standard (StandardSchema)

- le mapping entre ClickHouse et ce format (BIMappingConfig) ((correspondance) entre les colonnes d’une source BI (ClickHouse, Superset, PowerBI) et les colonnes du schéma standard)

et la fonction qui transforme les données ClickHouse en un format BI propre et harmonisé (standardize_clickhouse_data).

Imports

In [ ]:
import pandas as pd
from standardization import (
    standardize_clickhouse_data,
    BIMappingConfig,
    StandardSchema
)


Mapping + Schema

In [3]:
mapping = BIMappingConfig(
    source_tool="clickhouse",
    mapping={
        "date": "jour",
        "value": "valeur",
        "category": "type_vehicule",
        "kpi_title": "kpi"
    }
)

schema = StandardSchema()


**On crée le mapping et le schéma ici pour pouvoir les donner à la fonction `standardize_clickhouse_data`, qui en a besoin pour savoir comment traduire les colonnes ClickHouse vers le format BI standard.**

**Sans ces deux objets, la fonction ne saurait pas comment renommer les colonnes ni quel format final produire.**


TEST

In [4]:
from standardization import standardize_clickhouse_data

df_std = standardize_clickhouse_data(df_clickhouse_fake, mapping, schema)
print(df_std)
print(df_std.dtypes)


        date  value category kpi_title
0 2024-01-01   10.5      SUV    Ventes
1 2024-01-02   20.0  Berline    Ventes
date         datetime64[ns]
value               float64
category     string[python]
kpi_title    string[python]
dtype: object


La ligne avec "INVALID_DATE" a été supprimée ✔

Les types ont été normalisés ✔

Les colonnes ont été renommées ✔

                   (1) Data brute ClickHouse
      ┌─────────────────────────────────────────────┐
      │  jour | valeur | type_vehicule | kpi         │
      └─────────────────────────────────────────────┘
                          │
                          ▼
            2. Vérification du mapping & colonnes
                          │
                          ▼
            3. Renommage → format standard BI
                  jour → date
                  valeur → value
                  type_vehicule → category
                  kpi → kpi_title
                          │
                          ▼
          4. Normalisation des types de données
                date → datetime
                value → float
                category → string
                kpi_title → string
                          │
                          ▼
            5. Suppression des lignes invalides
                 (dates invalides, valeurs NaN)
                          │
                          ▼
           (6) Data standardisée (format unifié BI)
      ┌──────────────────────────────────────────────┐
      │   date | value | category | kpi_title        │
      └──────────────────────────────────────────────┘


Function: standardize_clickhouse_data(df, mapping_config, standard_schema)

INPUT:

    - df: raw ClickHouse dataframe
    - mapping_config: dict mapping ClickHouse columns → standard BI columns
    - standard_schema: standard BI schema (date, value, category, kpi_title)

STEPS:

    1. Check if df is empty → error
    2. Validate mapping columns exist
    3. Rename columns to standardized names
    4. Convert types:
         - date → datetime
         - value → float
         - category/title → string
    5. Remove invalid rows (NaT, NaN)
    6. Return standardized dataframe

OUTPUT:
    DataFrame with columns:
    [date, value, category, kpi_title]


In [10]:
print(df_std.to_string())
df_std = standardize_clickhouse_data(df_clickhouse_fake, mapping, schema)
df_std



        date  value category kpi_title
0 2024-01-01   10.5      SUV    Ventes
1 2024-01-02   20.0  Berline    Ventes


,date,value,category,kpi_title
0,2024-01-01,10.5,SUV,Ventes
1,2024-01-02,20.0,Berline,Ventes


Vérifier les types ( c’est BI-unifié)

In [9]:
print(df_std.dtypes)

date         datetime64[ns]
value               float64
category     string[python]
kpi_title    string[python]
dtype: object


Avant (ClickHouse brut) :


In [12]:
df_clickhouse_fake


,jour,valeur,type_vehicule,kpi
0,2024-01-01,10.5,SUV,Ventes
1,2024-01-02,20.0,Berline,Ventes
2,INVALID_DATE,30.0,SUV,Ventes


Après (Format BI unifié) :

In [13]:
df_std


,date,value,category,kpi_title
0,2024-01-01,10.5,SUV,Ventes
1,2024-01-02,20.0,Berline,Ventes


In [5]:
!pip install pytest
!pytest -vv


============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-4.12.0, langsmith-0.4.53, typeguard-4.4.4
collected 6 items                                                              

test_standardization.py::test_standardization_valid PASSED               [ 16%]
test_standardization.py::test_standardization_empty_df PASSED            [ 33%]
test_standardization.py::test_standardization_missing_column PASSED      [ 50%]
test_standardization.py::test_standardization_invalid_dates PASSED       [ 66%]
test_standardization.py::test_standardization_invalid_value PASSED       [ 83%]
test_standardization.py::test_standardization_types PASSED               [100%]

============================== 6 passed in 0.73s ===============================



| Tests manuels (avant) | Tester rapidement le code dans Colab |  Pour débugger          

| Fichier Pytest        | Tests automatiques professionnels     | Pour validation finale


In [14]:
%%writefile test_standardization.py
import pytest
import pandas as pd
from standardization import (
    standardize_clickhouse_data,
    BIMappingConfig,
    StandardSchema
)

# -------------------------------
# FIXTURES
# -------------------------------
# Les fixtures permettent de réutiliser facilement
# le même mapping, schéma et jeu de données
# dans tous les tests.

@pytest.fixture
def mapping():
    # Mapping ClickHouse → Schéma BI standardisé
    return BIMappingConfig(
        source_tool="clickhouse",
        mapping={
            "date": "jour",              # colonne standard 'date' correspond à 'jour' dans ClickHouse
            "value": "valeur",           # colonne standard 'value' correspond à 'valeur'
            "category": "type_vehicule", # colonne standard 'category' correspond à 'type_vehicule'
            "kpi_title": "kpi"           # colonne standard 'kpi_title' correspond à 'kpi'
        }
    )

@pytest.fixture
def schema():
    # Schéma BI final (colonnes standardisées)
    return StandardSchema()


@pytest.fixture
def clickhouse_ok():
    # Jeu de données ClickHouse valide
    # utilisé dans plusieurs tests
    return pd.DataFrame({
        "jour": ["2024-01-01", "2024-01-02"],
        "valeur": ["10.5", "20.0"],
        "type_vehicule": ["SUV", "Berline"],
        "kpi": ["Ventes", "Ventes"]
    })


# -------------------------------
# TESTS UNITAIRES
# -------------------------------

def test_standardization_valid(clickhouse_ok, mapping, schema):
    """
    Test principal : la standardisation doit produire :
    - les bonnes colonnes
    - la bonne longueur
    - les bonnes valeurs
    """
    df_std = standardize_clickhouse_data(clickhouse_ok, mapping, schema)

    expected_columns = ["date", "value", "category", "kpi_title"]

    # Vérifie que les colonnes ont été correctement renommées
    assert list(df_std.columns) == expected_columns

    # Vérifie que les 2 lignes valides sont conservées
    assert len(df_std) == 2

    # Vérifie la conversion en float et la bonne valeur
    assert df_std.iloc[0]["value"] == 10.5

    # Vérifie que la catégorie est correctement conservée
    assert df_std.iloc[1]["category"] == "Berline"


def test_standardization_empty_df(mapping, schema):
    """
    Si le DataFrame ClickHouse est vide → la fonction doit lever une ValueError.
    """
    df_empty = pd.DataFrame()

    with pytest.raises(ValueError):
        standardize_clickhouse_data(df_empty, mapping, schema)


def test_standardization_missing_column(clickhouse_ok, schema):
    """
    Si une colonne du mapping n'existe pas dans ClickHouse → KeyError.
    """
    # Mapping incorrect volontairement :
    wrong_mapping = BIMappingConfig(
        source_tool="clickhouse",
        mapping={"date": "jour", "value": "COLONNE_INEXISTANTE"}
    )

    with pytest.raises(KeyError):
        standardize_clickhouse_data(clickhouse_ok, wrong_mapping, schema)


def test_standardization_invalid_dates(mapping, schema):
    """
    Une date invalide doit être convertie en NaT puis supprimée.
    """
    df = pd.DataFrame({
        "jour": ["2024-01-01", "BAD_DATE"],  # BAD_DATE doit être éliminé
        "valeur": ["10.0", "20.0"],
        "type_vehicule": ["SUV", "SUV"],
        "kpi": ["Ventes", "Ventes"]
    })

    df_std = standardize_clickhouse_data(df, mapping, schema)

    # Une seule ligne valide doit rester
    assert len(df_std) == 1

    # Vérifie que c'est bien la bonne date
    assert df_std.iloc[0]["date"].strftime("%Y-%m-%d") == "2024-01-01"


def test_standardization_invalid_value(mapping, schema):
    """
    Une valeur non numérique doit être convertie en NaN puis supprimée.
    """
    df = pd.DataFrame({
        "jour": ["2024-01-01", "2024-01-02"],
        "valeur": ["10.0", "NOT_A_NUMBER"],  # Valeur invalide → supprimée
        "type_vehicule": ["SUV", "SUV"],
        "kpi": ["Ventes", "Ventes"]
    })

    df_std = standardize_clickhouse_data(df, mapping, schema)

    # Une seule ligne valide doit rester
    assert len(df_std) == 1

    # Vérifie la conversion correcte en float
    assert df_std.iloc[0]["value"] == 10.0


def test_standardization_types(clickhouse_ok, mapping, schema):
    """
    Vérifie que les types finaux respectent le format BI :
    - date : datetime
    - value : float
    - category : string
    - kpi_title : string
    """
    df_std = standardize_clickhouse_data(clickhouse_ok, mapping, schema)

    assert str(df_std["date"].dtype).startswith("datetime")
    assert df_std["value"].dtype == float
    assert df_std["category"].dtype.name == "string"
    assert df_std["kpi_title"].dtype.name == "string"


Overwriting test_standardization.py
